# CitySearch Web Scraping

## Implementation (8/20/24)

### Importing libraries

In [ ]:
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

### Navigating to the main page of CitySearch

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.citysearch.com/")

### Extracting the links to individual cities

In [ ]:
container = driver.find_element(By.CSS_SELECTOR, "div.cities-container")
cities = container.find_elements(By.CSS_SELECTOR, "li:not([class*='state']) > a")

city_links = [city.get_attribute("href") for city in cities]
state, city = re.search("(?<=\.com\/).*", city_links[0]).group().split("/") # extracting state and city name for later use

<img src="assets/first.png" alt="Alternative text" width="800" height="500"/>

### Navigating to a city link and gathering popular jobs

If we have keywords of specific industries we're interested in, I can iterate over them instead of iterating over popular industries. Also, if we have a list of states or cities we're interested in, I can also iterate over those.

In [ ]:
driver.get(city_links[0]) # as an example will be going through the jobs in first city
try:
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.categories-wrapper > ul > li > a")))
except TimeoutException:
    print("Timed out waiting for page to load")

popular_jobs = driver.find_elements(By.CSS_SELECTOR, 'div.categories-wrapper > ul > li > a')
popular_jobs_links = [job.get_attribute("href") for job in popular_jobs]

<img src="assets/second.png" alt="Alternative text" width="800" height="500"/>

### Navigating to first popular job and extracting links to jobs

In [ ]:
driver.get(popular_jobs_links[0])

try:
    elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.list-container > div.card > a")))
except TimeoutException:
    print("Timed out waiting for page to load")

job_cards = driver.find_elements(By.CSS_SELECTOR, "div.list-container > div.card > a")
job_cards_links = [job.get_attribute("href") for job in job_cards]

<img src="assets/third.png" alt="Alternative text" width="800" height="500"/>

### Scraping job description

In [ ]:
business_list = []

for i in range(0, 5): #5 should be job_cards_link's length when implemented fully

    driver.get(job_cards_links[i])

    try:
        elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.business-details")))
    except TimeoutException:
        print("Timed out waiting for page to load")

    # not sure if all business have all their contact info so creating a dictionary based on the class name and value
    business_details = driver.find_elements(By.CSS_SELECTOR, 'div.business-details > *')

    business_details_dict = {
        entry.get_attribute("class"): entry.text
        for entry in business_details
    }

    business_list.append(business_details_dict)
    time.sleep(3)

<img src="assets/fourth.png" alt="Alternative text" width="800" height="500"/>

### Converting to dataframe, renaming columns and exporting to csv

In [ ]:
df = pd.DataFrame.from_dict(business_list)

df.rename(columns={
    "business-name": "business name",
    "external-links-container": "external link",
    "phone-trigger": "phone number",
    "business-hours": "business hours"
}, inplace=True)

df.to_csv(f'./{state}_{city}.csv', index=False) # example al_birmingham.csv

In [ ]:
driver.quit()

#### Possible Improvements and Changes

Depending on the company's needs, I can store these values elsewhere instead of a CSV. Possibly in MongoDB or an SQL database.

When scraping large amounts of data, the current script may run into memory issues. During full implementation, I'll refactor the script into something more modular or OOP. Selenium has something called Page Object Model (POM). I'm not too familiar with POM but I am more than willing to try!

## Implementation (8/26/24)

### Importing Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import re

### Key for converting state name to acronym

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}


### Importing .xlxs file and filling in merged cells with previous value

In [ ]:
df = pd.read_excel("google_maps_keywords.xlsx")
df.loc[:, ["Country", "State"]] = df.loc[:, ["Country", "State"]].ffill()

### Opening CitySearch

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.citysearch.com/")

### Finding container for links of all cities and saving it to a variable

In [ ]:
container = driver.find_element(By.CSS_SELECTOR, "div.cities-container")
cities = container.find_elements(By.CSS_SELECTOR, "li:not([class*='state']) > a")
city_links = [city.get_attribute("href") for city in cities]
state, city = re.search("(?<=\.com\/).*", city_links[0]).group().split("/")


### Grouping dataframe by country and state

During implementation, I would look over all the countries and state. This would be outer most loop. Until implementation, I have hard coded the script to only loop over the cities in the US.

In [ ]:
grouped_df = df.groupby("Country")
grouped_countries = grouped_df.get_group("United States") #todo during implementation, instead of United States, it should be iterated
grouped_states = grouped_countries.groupby("State")

states = grouped_states.groups.keys()

### Looping for the states, cities, and industries in that order. 

Some cities didn't have any job postings for some industries, so I've added a TryExcept block to cover for those situations. Currently, it's only looping for the first 5 job postings, but during implementation, the loop would continue until it's extracted everything.

In [ ]:
for state in states:
    state_cities = grouped_states.get_group(state)["City"]
    state_industries = grouped_states.get_group(state)["Industry"]
    state_abbrev = us_state_to_abbrev[state]

    for city in state_cities:
        business_list = []

        if (pd.isnull(city)): continue

        for industry in state_industries:
            if (pd.isnull(industry)): continue

            url = f"https://www.citysearch.com/results?term={industry.strip().replace(' ', '%20')}&where={city.replace(' ', '%20')},%20{state_abbrev}"
            print("-------------------------", url, "----------------------------")
            driver.get(url)

            # extracting all links to jobs in this category
            try:
                elem = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.list-container > div.card > a")))
            except TimeoutException:
                print("Timed out waiting for page to load: Most likely no job listing in this category")

            job_cards = driver.find_elements(By.CSS_SELECTOR, "div.list-container > div.card > a")
            job_cards_links = [job.get_attribute("href") for job in job_cards]

            # visiting each job link for the current industry and scraping information

            if (len(job_cards_links) == 0): continue

            # for i in range(len(job_cards_links)): # todo uncomment this when implementing fully
            for i in range(5):
                print(job_cards_links[i])
                driver.get(job_cards_links[i])

                try:
                    elem = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.business-details")))
                except TimeoutException:
                    print("Timed out waiting for page to load")

                # not sure if all business have all their contact info so creating a dictionary based on the class name and value
                business_details = driver.find_elements(By.CSS_SELECTOR, 'div.business-details > *')

                business_details_dict = {
                    entry.get_attribute("class"): entry.text
                    for entry in business_details
                }
                business_details_dict["industry"] = industry

                business_list.append(business_details_dict)
                time.sleep(3)

        df = pd.DataFrame.from_dict(business_list)

        df.rename(columns={
            "business-name": "business name",
            "external-links-container": "external link",
            "phone-trigger": "phone number",
            "business-hours": "business hours"
        }, inplace=True)

        df.to_csv(f'./{state_abbrev.lower()}_{city.lower()}.csv', index=False)


In [ ]:
driver.quit()

## Results

<img src="assets/results.png" alt="Alternative text" width="" height="500"/>

#### I think it's ready for full implementation!